### **Data Mining Using SQL**

<font color="red">File access required:</font> In Colab this notebook requires first uploading files **Shop.csv** and **Movies.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure these files are in the same workspace as the notebook.

In [1]:
!pip install prettytable==0.7.2
!pip install ipython-sql

  Preparing metadata (setup.py) ... done
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13695 sha256=12f10da5562f888ce891c11cc4296ab7529d664bbc08e8302bf2c2a343d6f577
  Stored in directory: /root/.cache/pip/wheels/ca/f9/66/1ebeb8cdff2211eebb6fce02957f9e0a9ae3da4b7e65512d1b
Successfully built prettytable
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.17.0
    Uninstalling prettytable-3.17.0:
      Successfully uninstalled prettytable-3.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.1 MB/s eta 0:00:00


In [2]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [3]:
# Load tables from CSV files
# Shop
with open('Shop.csv') as f: Shop = pd.read_csv(f, index_col=0)
%sql drop table if exists Shop;
%sql --persist Shop
# Movies
with open('Movies.csv') as f: Movies = pd.read_csv(f, index_col=0)
%sql drop table if exists Movies;
%sql --persist Movies

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted movies'

## Shopping dataset - frequent item-sets

In [4]:
%%sql
select * from Shop

 * sqlite://
Done.


TID,item
1,milk
1,eggs
1,juice
2,milk
2,juice
2,cookies
3,eggs
3,chips
4,milk
4,eggs


### Frequent item-sets of two

#### All pairs of items occurring together

In [5]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item

 * sqlite://
Done.


item1,item2
eggs,milk
eggs,juice
juice,milk
juice,milk
cookies,milk
cookies,juice
chips,eggs
eggs,milk
juice,milk
cookies,milk


#### Now with number of transactions they occur together in

In [6]:
%%sql
select T1.item as item1, T2.item as item2, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item

 * sqlite://
Done.


item1,item2,count()
chips,cookies,1
chips,eggs,1
chips,juice,1
chips,milk,1
cookies,juice,2
cookies,milk,2
eggs,juice,1
eggs,milk,2
juice,milk,3


#### Total number of transactions

In [7]:
%%sql
select count(distinct TID) from Shop

 * sqlite://
Done.


count(distinct TID)
5


#### Put it together for frequent item-sets

In [8]:
support = .3
print(support)

0.3


In [ ]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2
cookies,juice
cookies,milk
eggs,milk
juice,milk


### Frequent item-sets of three

#### All triples of items occurring together

In [9]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item

 * sqlite://
Done.


item1,item2,item3
eggs,juice,milk
cookies,juice,milk
cookies,juice,milk
chips,juice,milk
chips,cookies,milk
chips,cookies,juice


#### Now with number of transactions they occur together in

In [10]:
%%sql
select distinct T1.item as item1, T2.item as item2, T3.item as item3, count()
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


item1,item2,item3,count()
chips,cookies,juice,1
chips,cookies,milk,1
chips,juice,milk,1
cookies,juice,milk,2
eggs,juice,milk,1


#### Put it together for frequent item-sets

In [11]:
support = .3
print(support)

0.3


In [12]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2,item3
cookies,juice,milk


### <font color = 'green'>**Your Turn - Movies dataset frequent item-sets**</font>

In [13]:
%%sql
select * from Movies limit 15

 * sqlite://
Done.


TID,item
145755,The Fault in Our Stars
145755,Boyhood
145755,Big Hero 6
145755,The Imitation Game
145755,Inside Out
117070,Inside Out
239040,Inside Out
177796,Gone Girl
177796,Magic Mike XXL
126875,The Imitation Game


In [14]:
%%sql
select count(distinct TID) as numUsers,
       count(distinct item) as numMovies
from Movies

 * sqlite://
Done.


numUsers,numMovies
1382,123


#### Mine for frequent item-sets of three and four items in the Movies dataset. Find a single support threshold where the number of frequent item-sets of three items is more than 10 but less than 20, and the number of frequent item-sets of four items is more than 0.

In [15]:
support = 0.04
print(support)

0.04


In [16]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3, count()
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support

 * sqlite://
Done.


item1,item2,item3,count()
Big Hero 6,Boyhood,Gone Girl,56
Big Hero 6,Boyhood,The Imitation Game,57
Big Hero 6,Gone Girl,Inside Out,85
Big Hero 6,Gone Girl,The Imitation Game,119
Big Hero 6,Inside Out,The Imitation Game,102
Boyhood,Gone Girl,The Imitation Game,95
Fury,Gone Girl,The Imitation Game,70
Gone Girl,Inside Out,The Imitation Game,103


In [17]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3, T4.item as item4, count()
from Movies T1, Movies T2, Movies T3, Movies T4
where T1.TID = T2.TID and T2.TID = T3.TID and T3.TID = T4.TID
and T1.item < T2.item and T2.item < T3.item and T3.item < T4.item
group by T1.item, T2.item, T3.item, T4.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support

 * sqlite://
Done.


item1,item2,item3,item4,count()
Big Hero 6,Gone Girl,Inside Out,The Imitation Game,59


## Shopping dataset - association rules

### Association rules with one item on the left-hand side

In [18]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules; include number of transactions

In [19]:
%%sql
select item, count() as numtrans
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### Do the same query but save the result in a table called "Frequents"

In [20]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [21]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans as LHS_count, count() as both
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,LHS_count,both
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2
juice,chips,3,1
juice,cookies,3,2
juice,eggs,3,1
juice,milk,3,3
milk,chips,4,1
milk,cookies,4,2
milk,eggs,4,2


#### Add check for specified confidence to get final result

In [22]:
%%sql
select T1.item as LHS, T2.item as RHS
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS,RHS
eggs,milk
juice,cookies
juice,milk
milk,juice


### Association rules with two items on the left-hand side

In [23]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [24]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [25]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for specified confidence to get final result

In [26]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS1,LHS2,RHS
juice,milk,cookies


## Shopping dataset - association rules with lift instead of confidence

#### Compute overall frequency of each item -- needed for lift calculation

In [27]:
%%sql
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item

 * sqlite://
Done.


item,1.0*count() / (select count(distinct TID) from Shop)
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


#### Do the same query but save the result in a table called "ItemFreq"

In [28]:
%%sql
drop table if exists ItemFreq;
create table ItemFreq(item, freq);
insert into ItemFreq
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item;
select * from ItemFreq

 * sqlite://
Done.
Done.
5 rows affected.
Done.


item,freq
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


### Association rules with one item on the left-hand side

In [29]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions.

In [30]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [31]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,numtrans,count()
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2
juice,chips,3,1
juice,cookies,3,2
juice,eggs,3,1
juice,milk,3,3
milk,chips,4,1
milk,cookies,4,2
milk,eggs,4,2


#### Add check for lift > 1 to get final result

In [32]:
%%sql
select T1.item as LHS, T2.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Frequent F, ItemFreq I
where T1.item = F.item
and T1.TID = T2.TID
and T2.item = I.item
and T1.item != T2.item
group by T1.item, T2.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS,RHS,lift
juice,cookies,1.6666666666666665
juice,milk,1.25
milk,cookies,1.25
milk,juice,1.25


### Association rules with two items on the left-hand side

In [33]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [34]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [35]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for lift > 1 to get final result

In [36]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Shop T3, Frequent F, ItemFreq I
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID and T3.item = I.item
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS1,LHS2,RHS,lift
juice,milk,cookies,1.6666666666666665


### <font color = 'green'>**Your Turn - Movies dataset association rules**</font>

#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and confidence thresholds (need not be the same) so the number of association rules is more than 10 but less than 20.


In [39]:
support = 0.015
confidence = 0.25
print(support, confidence)

0.015 0.25


In [41]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, count(*)
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > 0.015;

 * sqlite://
Done.
Done.
39 rows affected.


[]

In [42]:
%%sql
WITH ItemsetCounts AS (
    SELECT T1.item as i1, T2.item as i2, T3.item as i3, T4.item as i4,
           COUNT(*) as cnt
    FROM Movies T1, Movies T2, Movies T3, Movies T4
    WHERE T1.TID = T2.TID AND T2.TID = T3.TID AND T3.TID = T4.TID
    AND T1.item < T2.item AND T2.item < T3.item AND T3.item < T4.item
    GROUP BY T1.item, T2.item, T3.item, T4.item
    HAVING 1.0 * COUNT(*) / (SELECT COUNT(DISTINCT TID) FROM Movies) > 0.015
)
SELECT
    ic.i1 || ', ' || ic.i2 || ', ' || ic.i3 AS LHS,
    ic.i4 AS RHS,
    1.0 * ic.cnt / F.numtrans AS confidence,
    1.0 * ic.cnt / (SELECT COUNT(DISTINCT TID) FROM Movies) AS support
FROM ItemsetCounts ic
JOIN Frequent F ON ic.i1 = F.item1 AND ic.i2 = F.item2 AND ic.i3 = F.item3
WHERE 1.0 * ic.cnt / F.numtrans >= 0.25
ORDER BY confidence DESC, support DESC;

 * sqlite://
Done.


LHS,RHS,confidence,support
"Big Hero 6, Boyhood, Fury",The Imitation Game,0.9655172413793104,0.020260492040520984
"Boyhood, Fury, Inside Out",The Imitation Game,0.9583333333333334,0.016642547033285094
"Big Hero 6, Boyhood, Fury",Gone Girl,0.9310344827586207,0.019536903039073805
"Boyhood, Fury, Gone Girl",The Imitation Game,0.9069767441860465,0.028219971056439943
"Fury, Gone Girl, Inside Out",The Imitation Game,0.9,0.02604920405209841
"Big Hero 6, Boyhood, Inside Out",The Imitation Game,0.8974358974358975,0.02532561505065123
"Big Hero 6, Fury, Gone Girl",The Imitation Game,0.8372093023255814,0.02604920405209841
"Big Hero 6, Fury, Inside Out",The Imitation Game,0.8235294117647058,0.020260492040520984
"Big Hero 6, Boyhood, Gone Girl",The Imitation Game,0.8214285714285714,0.03328509406657019
"Boyhood, Gone Girl, Inside Out",The Imitation Game,0.7777777777777778,0.030390738060781478


#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and lift thresholds so the number of association rules is more than 10 but less than 20.


In [43]:
support = 0.01
lift = 0.1
print(support, lift)

0.01 0.1


In [44]:
%%sql
drop table if exists ItemFreq;
create table ItemFreq as
select item,
       count(distinct TID) as freq,
       1.0 * count(distinct TID) / (select count(distinct TID) from Movies) as item_support
from Movies
group by item;

 * sqlite://
Done.
Done.


[]

In [45]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, count(*)
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > 0.01;

 * sqlite://
Done.
Done.
81 rows affected.


[]

In [46]:
%%sql
WITH ItemsetCounts AS (
    SELECT T1.item as i1, T2.item as i2, T3.item as i3, T4.item as i4,
           COUNT(*) as cnt
    FROM Movies T1, Movies T2, Movies T3, Movies T4
    WHERE T1.TID = T2.TID AND T2.TID = T3.TID AND T3.TID = T4.TID
    AND T1.item < T2.item AND T2.item < T3.item AND T3.item < T4.item
    GROUP BY T1.item, T2.item, T3.item, T4.item
    HAVING 1.0 * COUNT(*) / (SELECT COUNT(DISTINCT TID) FROM Movies) > 0.01
)
SELECT
    ic.i1 || ', ' || ic.i2 || ', ' || ic.i3 AS LHS,
    ic.i4 AS RHS,
    1.0 * ic.cnt / (SELECT COUNT(DISTINCT TID) FROM Movies) AS support,
    1.0 * ic.cnt / F.numtrans AS confidence,
    (1.0 * ic.cnt / (SELECT COUNT(DISTINCT TID) FROM Movies)) / if4.item_support AS lift
FROM ItemsetCounts ic
JOIN Frequent F ON ic.i1 = F.item1 AND ic.i2 = F.item2 AND ic.i3 = F.item3
JOIN ItemFreq if4 ON ic.i4 = if4.item
WHERE (1.0 * ic.cnt / (SELECT COUNT(DISTINCT TID) FROM Movies)) / if4.item_support >= 0.1
ORDER BY lift DESC, support DESC;

 * sqlite://
Done.


LHS,RHS,support,confidence,lift
"Boyhood, Gone Girl, The Imitation Game",Wild Tales,0.015918958031837915,0.23157894736842105,0.32352941176470584
"Gone Girl, Inside Out, The Imitation Game",Wild Tales,0.013024602026049204,0.17475728155339806,0.2647058823529412
"Big Hero 6, Gone Girl, Inside Out",Wild Tales,0.011577424023154847,0.18823529411764706,0.2352941176470588
"Big Hero 6, Gone Girl, The Imitation Game",Wild Tales,0.01085383502170767,0.12605042016806722,0.22058823529411764
"Big Hero 6, Inside Out, The Imitation Game",Wild Tales,0.01085383502170767,0.14705882352941177,0.22058823529411764
"Fury, Gone Girl, The Imitation Game",Wild Tales,0.01085383502170767,0.21428571428571427,0.22058823529411764
"Big Hero 6, Fury, The Imitation Game",Transformers: Age of Extinction,0.01085383502170767,0.3409090909090909,0.21739130434782605
"Big Hero 6, Boyhood, The Imitation Game",Wild Tales,0.010130246020260492,0.24561403508771928,0.20588235294117646
"Fury, Gone Girl, The Imitation Game",Transformers: Age of Extinction,0.010130246020260492,0.2,0.20289855072463767
"Big Hero 6, Gone Girl, Inside Out",Louis C.K.: Live at The Comedy Store,0.010130246020260492,0.16470588235294117,0.15555555555555556
